# Install libraries

In [1]:
# install libraries
!pip install catboost


^C
ERROR: Operation cancelled by user


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import plotly.express as px
from IPython.display import display

# score metrics and splitting libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

# ML algorithms from sklearn
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor

# extreme algorithms
from xgboost import XGBRFRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# warning turn off
import warnings
warnings.filterwarnings('ignore')

In [3]:
#set display max column to see all column
pd.set_option("display.max_columns", None)
pd.get_option("display.max_columns")

#set display max column to see all column
pd.set_option("display.max_rows", 150)
pd.get_option("display.max_rows")

150

# Load Datasets

In [4]:
# load datasets

train = pd.read_csv('/kaggle/input/delays-data-cleaned/cleaned_train.csv')
test = pd.read_csv('/kaggle/input/delays-data-cleaned/cleaned_test.csv')

train.head()

,Weekday,Month_of_Year,Day_of_Month,Departure_State,Arrival_State,Flight_Cancelled,Departure_Delay,Arrival_Delay,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation,Scheduled_Departure_Time_Minutes,Scheduled_Arrival_Time_Minutes,Actual_Departure_Time_Minutes
0,2,6,21,3,3,0,0.088687,-4.178483,16.0,5.0,0,23.0,110.0,13.279939,20.470690,18.045064,12.910265,0.0,0.0,736,784,736
1,3,1,5,22,50,0,13.027852,5.042185,34.0,5.0,0,36.0,119.0,17.816202,13.967273,21.606228,17.976362,0.0,0.1,1185,1255,1185
2,2,3,22,8,44,0,-1.802698,-0.206932,10.0,11.0,0,125.0,641.0,24.562566,14.509228,24.946489,22.630553,0.0,0.0,420,564,418
3,4,7,14,49,32,0,13.027852,14.006092,23.0,7.0,0,130.0,867.0,8.817992,10.866812,17.426336,17.401007,0.0,0.0,1290,1439,1293
4,1,5,30,4,4,0,-2.181755,-14.067374,10.0,5.0,0,58.0,417.0,9.360464,15.977111,19.001179,16.884960,0.0,0.0,470,555,468


We split the train, contains target variable, to train and validation to train and test models performance and accuracy.

In [5]:
# split train to train set and validation sets, use 75 - 25 % approach
# Shuffle the indices
indices = np.arange(len(train))
np.random.shuffle(indices)

# Define the split ratio
split_ratio = 0.75

# Calculate split indices
split_index = int(len(train) * split_ratio)

# Split the data
df_train = train.iloc[indices[:split_index], :]
df_test = train.iloc[indices[split_index:], :]

df_train.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

print(f'Train size : {df_train.shape}')
print(f'Test size : {df_test.shape}')

Train size : (1065015, 22)
Test size : (355006, 22)


In [6]:
display(df_train.head())

print('*' * 70)
df_test.head()

,Weekday,Month_of_Year,Day_of_Month,Departure_State,Arrival_State,Flight_Cancelled,Departure_Delay,Arrival_Delay,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation,Scheduled_Departure_Time_Minutes,Scheduled_Arrival_Time_Minutes,Actual_Departure_Time_Minutes
0,2,3,15,13,49,0,-1.051708,-11.996470,16.0,10.0,0,243.0,799.232715,16.596518,16.598338,26.629680,28.691390,0.1,0.0,785,945,784
1,1,6,13,1,44,0,17.361262,9.934543,10.0,17.0,0,87.0,603.000000,15.394063,14.400558,24.233927,20.574877,0.0,0.0,759,880,776
2,2,4,19,18,4,0,13.027852,-7.031112,22.0,6.0,0,373.0,2704.000000,21.857150,26.405182,26.901584,31.157041,0.5,0.0,720,943,715
3,2,4,5,37,48,0,65.171736,53.184246,16.0,3.0,0,54.0,335.000000,10.002911,9.072254,19.935944,16.370793,0.0,1.0,830,915,895
4,2,2,15,26,46,0,-9.697178,-13.806691,13.0,3.0,0,57.0,322.000000,20.534626,17.550406,20.735905,14.342107,0.0,0.0,1085,1162,1075


**********************************************************************


,Weekday,Month_of_Year,Day_of_Month,Departure_State,Arrival_State,Flight_Cancelled,Departure_Delay,Arrival_Delay,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation,Scheduled_Departure_Time_Minutes,Scheduled_Arrival_Time_Minutes,Actual_Departure_Time_Minutes
0,6,6,18,32,23,0,4.998664,-1.028720,14.0,11.0,0,165.00000,1371.0,14.790753,15.358523,22.614930,22.847892,0.0,0.0,405,721,410
1,7,6,5,13,4,0,18.731438,10.958971,10.0,3.0,0,111.10592,1728.0,20.971280,16.038952,20.077378,20.082186,0.0,0.0,820,945,839
2,4,3,3,26,40,0,-4.149163,-17.124410,18.0,3.0,0,111.10592,157.0,11.872586,14.298393,20.077378,20.082186,0.1,0.0,820,886,816
3,7,7,31,44,44,0,-5.359531,-11.332187,10.0,7.0,0,33.00000,190.0,16.575124,11.454507,18.501859,17.156482,0.0,1.0,1260,1317,1255
4,5,5,27,23,13,0,23.987912,19.979432,9.0,6.0,0,56.00000,405.0,12.187095,9.717764,18.925081,14.814462,0.0,0.0,1310,1385,1334


# Parameter Tuning

When working with a large dataset like ours (1 million rows), finding the best parameters for regression models while ensuring efficient model training and evaluation cost to much computation and time.

To handle this issue we use **Sampling for Hyperparameter Tuning**. It means that instead of using the entire dataset for hyperparameter tuning, we can use a representative sample of data. This will significantly reduce computation time and allow us to perform thorough hyperparameter tuning.

In [7]:
# use 15% of the data for tuning
sample_size = int(0.15 * len(df_train))
df_sample = df_train.sample(n = sample_size, random_state = 42)

X_sample = df_sample.drop(columns=['Arrival_Delay'])
y_sample = df_sample['Arrival_Delay']

## Parameters for baseline models

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size = 0.3, random_state = 0)

X_train.shape

(111826, 21)

In [9]:
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error


# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


In [11]:
# Define the search space for hyperparameters
# space = {
#     'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1.0),
#     'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
#     'random_strength': hp.uniform('random_strength', 0.01, 0.1),
#     'max_bin': hp.quniform('max_bin', 100, 1000, 10),
#     'depth': hp.randint('depth', 3, 10),  # integers from 3 to 9
#     'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
#     'bootstrap_type': 'Bernoulli', 
#     'subsample': hp.uniform('subsample', 0.7, 0.9),
# }

space = {
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1.0),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'random_strength': hp.uniform('random_strength', 0.01, 0.1),
    'max_bin': hp.quniform('max_bin', 100, 1000, 10),
    'depth': hp.randint('depth', 3, 10),  # integers from 3 to 9
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
    'bootstrap_type': 'Bernoulli',  # Fixed value
    'subsample': hp.uniform('subsample', 0.7, 0.9),
    'iterations': 1000,
    'eval_metric': 'MAPE',
    'loss_function': 'MAPE',
    'random_seed': 42
}


In [14]:
from catboost import CatBoostRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_absolute_error, mean_squared_error

def objective(params):
    # Convert max_bin to integer
    params['max_bin'] = int(params['max_bin'])
    params['depth'] = int(params['depth'])
    
    # Create the CatBoost regressor
    model = CatBoostRegressor(**params, silent=True)
    
    evaluation = [(X_train, y_train), (X_test, y_test)]
    
    
    # Train the model
    model.fit(X_train, y_train, early_stopping_rounds=20, eval_set = evaluation, verbose = False)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate Mean Absolute Error
    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    print("MAPE:", mape)
    
    return {'loss': mape, 'status': STATUS_OK}


In [16]:
# Initialize trials object
trials = Trials()

# Perform hyperparameter tuning using fmin and TPE
best = fmin(fn=objective,
            space=space, 
            algo=tpe.suggest, 
            max_evals=100, 
            trials=trials)

# Print the best hyperparameters
print("Best hyperparameters:", best)


MAPE:                                                  
10.252452997024514                                     
MAPE:                                                                            
10.982138719897021                                                               
MAPE:                                                                            
10.295742571267889                                                               
MAPE:                                                                            
12.328498522387813                                                               
MAPE:                                                                            
9.349575092939522                                                                
MAPE:                                                                            
10.642146234777398                                                              
MAPE:                                                                

In [18]:
print("The best hyperparameters are : ","\n")
print(best)

The best hyperparameters are :  

{'colsample_bylevel': 0.6312979928865734, 'depth': 9, 'l2_leaf_reg': 8.754132470161709, 'learning_rate': 0.09396165335017637, 'max_bin': 490.0, 'random_strength': 0.07403229688797397, 'subsample': 0.8801936829680834}


In [19]:
best_parameters = best
best_parameters

{'colsample_bylevel': 0.6312979928865734,
 'depth': 9,
 'l2_leaf_reg': 8.754132470161709,
 'learning_rate': 0.09396165335017637,
 'max_bin': 490.0,
 'random_strength': 0.07403229688797397,
 'subsample': 0.8801936829680834}

In [20]:
# test model with using big data
X_train = df_train.drop(columns=['Arrival_Delay'])
X_test = df_test.drop(columns=['Arrival_Delay'])
y_train = df_train['Arrival_Delay']
y_test = df_test['Arrival_Delay']

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1065015, 21) (1065015,)
(355006, 21) (355006,)


In [22]:
model = CatBoostRegressor(**best_parameters, silent=True)
    
evaluation = [(X_train, y_train), (X_test.sample(n = sample_size, random_state = 42), y_test.sample(n = sample_size, random_state = 42))]
    
for set in evaluation:
    print(set[0].shape, set[1].shape)

(1065015, 21) (1065015,)
(159752, 21) (159752,)


In [26]:
    
# Train the model
model.fit(X_train, y_train, early_stopping_rounds=20, eval_set = evaluation, verbose = False)
    
# Make predictions
y_pred = model.predict(X_test)
    
# Calculate Mean Absolute Error
mape = mean_absolute_percentage_error(y_test, y_pred)



In [27]:
# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MAPE :', mape)

MAPE : 16.401544716817433


In [25]:
model1 = CatBoostRegressor(**best_parameters, silent=True)

# Train the model
model1.fit(X_train, y_train)
    
# Make predictions
y_pred = model1.predict(X_test)
    
# Calculate Mean Absolute Error
mape = mean_absolute_percentage_error(y_test, y_pred)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MAPE :', mape)

MAPE : 16.401544716817433


### K Fold cross validation

In [28]:
%%time
# cross validation technique

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error

# Create the XGBRegressor model with the best parameters
cat_reg1 = CatBoostRegressor(**best_parameters, silent=True)


# Define the K-Fold cross-validation strategy
kfold = KFold(n_splits=7, shuffle=True, random_state=42)

# Initialize lists to store evaluation scores
mape_scores = []

# Perform K-Fold Cross-Validation
for train_index, test_index in kfold.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    # Fit the XGBoost model on the training fold
    cat_reg1.fit(X_train_fold, y_train_fold)

    # Predict on the validation fold
    y_pred_val = cat_reg1.predict(X_val_fold)

    # Calculate MAPE on the validation fold
    mape = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    print('folds are working : ', mape)
    mape_scores.append(mape)

# Calculate mean MAPE across all folds
mean_mape = sum(mape_scores) / len(mape_scores)
print("Mean MAPE:", mean_mape)



folds are working :  16.595545434780558
folds are working :  6.943370782631499
folds are working :  5.41708629388723
folds are working :  13.398933550825271
folds are working :  6.5964899760237845
folds are working :  9.545053487538263
folds are working :  16.41784946240237
Mean MAPE: 10.702046998298425
CPU times: user 1h 14min 40s, sys: 4min 46s, total: 1h 19min 26s
Wall time: 23min


In [31]:
cat_reg1.get_params()

{'learning_rate': 0.09396165335017637,
 'depth': 9,
 'l2_leaf_reg': 8.754132470161709,
 'loss_function': 'RMSE',
 'silent': True,
 'random_strength': 0.07403229688797397,
 'subsample': 0.8801936829680834,
 'colsample_bylevel': 0.6312979928865734,
 'max_bin': 490.0}

In [32]:
the_best_params = {'learning_rate': 0.09396165335017637,
 'depth': 9,
 'l2_leaf_reg': 8.754132470161709,
 'loss_function': 'MAPE',
 'silent': True,
 'random_strength': 0.07403229688797397,
 'subsample': 0.8801936829680834,
 'colsample_bylevel': 0.6312979928865734,
 'max_bin': 490.0}

In [34]:
cat_reg2 = CatBoostRegressor(**the_best_params)


# Train the model
cat_reg2.fit(X_train, y_train)
    
# Make predictions
y_pred = cat_reg2.predict(X_test)
    
# Calculate Mean Absolute Error
mape = mean_absolute_percentage_error(y_test, y_pred)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MAPE :', mape)

MAPE : 28.04523187700711
